In [3]:
%pip install pyproj

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----------- ---------------------------- 1.8/6.3 MB 11.2 MB/s eta 0:00:01
   -------------------------- ------------- 4.2/6.3 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 10.5 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from pyproj import Transformer

# =========================
# 설정
# =========================
INPUT_CSV  = "seoul_sg.csv"          # <- 원본 파일명으로 바꿔줘
OUTPUT_CSV = "signals_passenger.csv"       # <- 결과 파일명

# 보통 한국 TM(중부원점) 좌표가 EPSG:5186인 경우가 많음
SRC_EPSG = "EPSG:5186"
DST_EPSG = "EPSG:4326"

# 원본에 '시도' 컬럼이 없으면 여기서 기본값 지정(강남구면 보통 서울특별시)
DEFAULT_SIDO = "서울특별시"

# 도로종류(한글) → road_type 코드(메타데이터 기준)
# 03:특별시도, 05:시도, 07:구도, 99:기타 등
ROAD_TYPE_MAP = {
    "고속국도": 1,
    "일반국도": 2,
    "특별시도": 3,
    "지방도": 4,
    "시도": 5,
    "군도": 6,
    "구도": 7,
}

# 신호등종류(텍스트) → signal_type 코드(간단 규칙)
# (원본 텍스트가 다양해서, "보행" 포함이면 02(보행등), 그 외는 99로 둠)
def map_signal_type(text: str) -> int:
    t = str(text)
    if "보행" in t:
        return 2
    if "차량" in t:
        return 1
    if "버스" in t:
        return 3
    if "자전거" in t:
        return 4
    if "경보" in t:
        return 6
    if "가변" in t:
        return 7
    return 99

def yn_to_10(v) -> int:
    s = str(v).strip()
    if s in ("유", "Y", "y", "1", "true", "True"):
        return 1
    if s in ("무", "N", "n", "0", "false", "False"):
        return 0
    return 0  # 애매하면 0 처리

# =========================
# 로드
# =========================
df = pd.read_csv(INPUT_CSV, encoding="cp949")

# 컬럼명 공백/개행 정리(엑셀에서 저장하면 종종 이상해짐)
df.columns = [c.strip().replace("\n", "").replace("\r", "") for c in df.columns]

# =========================
# 좌표 변환 (X,Y → lon,lat)
# =========================
transformer = Transformer.from_crs(SRC_EPSG, DST_EPSG, always_xy=True)

x = pd.to_numeric(df["X좌표"], errors="coerce")
y = pd.to_numeric(df["Y좌표"], errors="coerce")

lon, lat = transformer.transform(x.to_numpy(), y.to_numpy())

# =========================
# signals_passenger 스키마로 매핑
# =========================
out = pd.DataFrame()

# sido: 원본에 없으니 기본값 / 있으면 사용
out["sido"] = df["시도"].astype(str) if "시도" in df.columns else DEFAULT_SIDO

# sigungu: 시군구명 → sigungu
out["sigungu"] = df["시군구명"].astype(str)

# road_type: 도로종류 → 코드
out["road_type"] = df["도로종류"].map(ROAD_TYPE_MAP).fillna(99).astype(int)

# road_direction: 원본에 없으니 03(양방향) 기본
out["road_direction"] = 3

# address: 주소
out["address"] = df["주소"].astype(str)

# 위경도
out["signal_lat"] = lat
out["signal_lon"] = lon

# road_shape: 원본에 없으니 99(기타) 기본
out["road_shape"] = 99

# main_road: 원본에 없으니 0 기본(주도로 여부)
out["main_road"] = 0

# signal_type: 신호등종류 텍스트 → 코드
out["signal_type"] = df["신호등종류"].apply(map_signal_type).astype(int)

# button/remain_time/sound_signal: 유/무 → 1/0
# 원본 컬럼명이 정확히 아래와 다르면, print(df.columns)로 확인 후 바꿔줘
out["button"] = df["보행자작동신호기유무"].apply(yn_to_10).astype(int)
out["remain_time"] = df["잔여시간표시장치유무"].apply(yn_to_10).astype(int)
out["sound_signal"] = df["음향신호기유무"].apply(yn_to_10).astype(int)

# org_code: 원본에 없으니 NaN(또는 0)로
out["org_code"] = np.nan

# sg_uid: 관리번호를 기본 ID로 사용하되, 중복이면 suffix 부여
base_id = df["관리번호"].astype(str).str.strip()
dup_rank = base_id.groupby(base_id).cumcount() + 1
out["sg_uid"] = np.where(dup_rank == 1, base_id, base_id + "_" + dup_rank.astype(str))

# =========================
# 품질 점검(권장)
# =========================
# 1) 위경도 범위 체크(한국 대략)
bad_geo = ~out["signal_lat"].between(33, 39) | ~out["signal_lon"].between(124, 132)
if bad_geo.any():
    print("⚠️ 위경도 범위가 이상한 행이 있습니다. SRC_EPSG(기본 5186)가 맞는지 확인하세요.")
    print(out.loc[bad_geo, ["address", "signal_lat", "signal_lon"]].head(10))

# 2) sg_uid 유니크 체크
if out["sg_uid"].duplicated().any():
    print("⚠️ sg_uid 중복이 남아있습니다. 생성 규칙을 조정하세요.")

# =========================
# 저장
# =========================
out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ saved: {OUTPUT_CSV}  rows={len(out)}")
print(out.head(3))


✅ saved: signals_passenger.csv  rows=65001
  sido sigungu  road_type  road_direction       address  signal_lat  \
0  NaN     강남구          5               3   강남구 개포동 10천   37.491514   
1  NaN     강남구          5               3  강남구 개포동 12 대   37.492734   
2  NaN     강남구          5               3  강남구 개포동 12 대   37.492734   

   signal_lon  road_shape  main_road  signal_type  button  remain_time  \
0  127.072462          99          0            2       0            1   
1  127.076054          99          0            2       0            1   
2  127.076054          99          0            2       0            1   

   sound_signal  org_code           sg_uid  
0             1       NaN    03-0000066066  
1             1       NaN    03-0000014468  
2             1       NaN  03-0000014468_2  
